In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper Parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [3]:
# MNIST dataset
train_dataset = dsets.MNIST(root='/home/damvantai/Documents/data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
test_dataset = dsets.MNIST(root='/home/damvantai/Documents/data/',
                          train=False,
                          transform=transforms.ToTensor())

In [4]:
# Data Loader (inputpipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [5]:
# BiRNN model (many to one)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
    
    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        # 2 for bidirection
        c0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size))
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

In [6]:
rnn = BiRNN(input_size, hidden_size, num_layers, num_classes)

In [7]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr = learning_rate)

In [8]:
# Training the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, sequence_length, input_size))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print("Epoch [%d/%d], Step [%d/%d], loss: %.4f" % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/2], Step [100/600], loss: 0.6913
Epoch [1/2], Step [200/600], loss: 0.3553
Epoch [1/2], Step [300/600], loss: 0.2066
Epoch [1/2], Step [400/600], loss: 0.0709
Epoch [1/2], Step [500/600], loss: 0.0694
Epoch [1/2], Step [600/600], loss: 0.2505
Epoch [2/2], Step [100/600], loss: 0.0706
Epoch [2/2], Step [200/600], loss: 0.0465
Epoch [2/2], Step [300/600], loss: 0.0623
Epoch [2/2], Step [400/600], loss: 0.0731
Epoch [2/2], Step [500/600], loss: 0.0812
Epoch [2/2], Step [600/600], loss: 0.0691


In [9]:
# Test the model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

In [10]:
print('Test Accuracy of the model on the 10000, test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000, test images: 97 %


In [11]:
torch.save(rnn.state_dict(), '/home/damvantai/Documents/data/pytorch_bidirectional_rnn.pkl')